In [1]:
print("Importing torch...")
import torch
import torch.nn as nn
print("Importing HF...")
from transformers import AutoTokenizer, AutoModelForCausalLM
print("Importing python modules...")
from timehelp import time_start, time_end
from model_wrapper import Model, ModelFamily, MultipleChoiceStrategy
import re
print("Done!")

Importing torch...
Importing HF...
Importing python modules...
Done!


In [12]:
model = Model(ModelFamily.CodeGen1.multi["16B"])
model.configure(time=True)

[2024-05-14@19:37:33|model.device] Starting timer.
Configuring torch device...
Using device: cuda:0 aka cuda:0
[2024-05-14@19:37:33|model.device] Time elapsed: ~0s
[2024-05-14@19:37:33|model.tokenizer] Starting timer.


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

[2024-05-14@19:37:33|model.tokenizer] Time elapsed: 686ms
[2024-05-14@19:37:33|model.model] Starting timer.
Obtaining model...


config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/32.2G [00:00<?, ?B/s]

[2024-05-14@19:48:52|model.model] Time elapsed: 11min 18s


In [13]:
buggy = "private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this , VAR_1 ) ; METHOD_2 ( intent ) ; } "
fixed = "public void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this , VAR_1 ) ; METHOD_2 ( intent ) ; } "
print(buggy)
print(fixed)

private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this , VAR_1 ) ; METHOD_2 ( intent ) ; } 
public void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this , VAR_1 ) ; METHOD_2 ( intent ) ; } 


In [15]:
prompt = "// code before bug fix\n{code}\n// fixed code\n"
tokens = model.generate(prompt.format(code=buggy), time=True, max_length=128)
print(model.decode(tokens))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2024-05-14@19:58:04|model.tokenize] Starting timer.
[2024-05-14@19:58:04|model.tokenize] Time elapsed: 106ms
Generating...
[2024-05-14@19:58:04|model.generate] Starting timer.
[2024-05-14@20:03:45|model.generate] Time elapsed: 5min 41s
// code before bug fix
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
// fixed code
private void METHOD_2 ( android.content.Intent intent ) { android.content.Intent intent1 = new android.content.Intent ( this, android.content.Intent.ACTION_VIEW ) ; intent1.setData ( android.net.Uri.parse ( "http://


In [16]:
prompt = "// code before bug fix\n{code}\n"
tokens = model.generate(prompt.format(code=buggy), time=True, max_length=128)
print(model.decode(tokens))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2024-05-14@20:07:12|model.tokenize] Starting timer.
[2024-05-14@20:07:12|model.tokenize] Time elapsed: 106ms
Generating...
[2024-05-14@20:07:12|model.generate] Starting timer.
[2024-05-14@20:13:15|model.generate] Time elapsed: 6min 2s
// code before bug fix
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
private void METHOD_2 ( android.content.Intent intent ) { intent.setClass ( this, VAR_1 ) ; } 
// end of method

// method: METHOD_3
// sig: (Ljava/lang/Class;)V
// desc: METHOD_3
public


In [17]:
prompt = "// TODO: Fix this code\n{code}\n"
tokens = model.generate(prompt.format(code=buggy), time=True, max_length=128)
print(model.decode(tokens))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2024-05-14@20:20:20|model.tokenize] Starting timer.
[2024-05-14@20:20:20|model.tokenize] Time elapsed: 106ms
Generating...
[2024-05-14@20:20:20|model.generate] Starting timer.
[2024-05-14@20:26:13|model.generate] Time elapsed: 5min 52s
// TODO: Fix this code
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
private void METHOD_2 ( android.content.Intent intent ) { intent.setFlags ( android.content.Intent.FLAG_ACTIVITY_NEW_TASK ) ; intent.setFlags ( android.content.Intent.FLAG_ACTIVITY_CLEAR_TOP ) ; startActivity


In [18]:
prompt = "{code}\n"
tokens = model.generate(prompt.format(code=buggy), time=True, max_length=128)
print(model.decode(tokens))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2024-05-14@20:26:35|model.tokenize] Starting timer.
[2024-05-14@20:26:35|model.tokenize] Time elapsed: 106ms
Generating...
[2024-05-14@20:26:35|model.generate] Starting timer.
[2024-05-14@20:33:10|model.generate] Time elapsed: 6min 34s
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
    private void METHOD_2 ( android.content.Intent intent ) { android.content.Intent intent2 = new android.content.Intent ( this, android.content.Intent.ACTION_VIEW ) ; intent2.setData ( android.net.Uri.parse ( "http://www.google.com" ) ) ;


In [20]:
prompt = "// the buggy version of the code\n{code}\n// the fixed version of the code\n"
tokens = model.generate(prompt.format(code=buggy), time=True, max_length=128)
print(model.decode(tokens))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[2024-05-14@20:49:10|model.tokenize] Starting timer.
[2024-05-14@20:49:10|model.tokenize] Time elapsed: 1ms
Generating...
[2024-05-14@20:49:10|model.generate] Starting timer.
[2024-05-14@20:54:21|model.generate] Time elapsed: 5min 10s
// the buggy version of the code
private void METHOD_1 ( java.lang.Class VAR_1 ) { android.content.Intent intent = new android.content.Intent ( this, VAR_1 ) ; METHOD_2 ( intent ) ; } 
// the fixed version of the code
private void METHOD_2 ( android.content.Intent intent ) { intent.setClass ( this, VAR_1 ) ; } 
// the fixed version of the code
private void METHOD_3 ( android.content.Intent intent ) { intent.set
